In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce
from scipy import stats
import re
import warnings
from geopy.geocoders import ArcGIS
from geopy.distance import geodesic
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('punkt')
nltk.download('stopwords')
warnings.filterwarnings( 'ignore' )
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pliku\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pliku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
hotels = pd.read_csv('data/hotels.csv')

## 1. Подготовка и очистка данных

* Избавление от дубликатов

In [3]:
print(f'Изначальное количество записей: {hotels.shape[0]}')
mask = hotels.duplicated()
print(f'Количество дубликатов: {hotels[mask].shape[0]}')
hotels = hotels.drop_duplicates()
print(f'Количество записей после очистки: {hotels.shape[0]}')

Изначальное количество записей: 386803
Количество дубликатов: 307
Количество записей после очистки: 386496


* Поиск и устранение неинформативных признаков

In [4]:
# список для имён малоинформативных признаков
low_info = []

# цикл пройдёт по столбцам, вычислит долю уникальных значений и
# долю одинаковых значений в каждом признаке
for i in hotels.columns:
    same = hotels[i].value_counts(normalize=True).max()
    unique = hotels[i].nunique() / hotels[i].count()
    if same > 0.95:
        low_info.append(i)
        print(f'{i}: {same}')
    if unique > 0.95:
        low_info.append(i)
        print(f'{i}: {unique}')
        
print('Количество признаков с долей уникальных и однинаковых\n'
    f'записей > 0.95: {len(low_info)}')

Количество признаков с долей уникальных и однинаковых
записей > 0.95: 0


# 2. Проектирование признаков

* Признак государственной принадлежности отеля

In [5]:
# уникальные значения стран
hotels['hotel_country'] = hotels['hotel_address'].apply(
    lambda x: 'United ' + x.split()[-1] if x.split()[-1] == 'Kingdom' else x.split()[-1]
    )


* Создание бинарного признака: 
*Является-ли ревьюер гражданином страны в которой расположен отель?*

In [6]:
hotels['is_native'] = hotels.apply(
    lambda x: 1 if (x['reviewer_nationality']).strip() == x['hotel_country'] else 0,
    axis=1
    )

* Работа с признаком "тэги"

In [7]:
# список тэгов
tag_list = ['a']

# цикл для извлечения тэгов из признака
for i in hotels['tags'].to_list():
    tag_list.extend(i.replace("[' ", "").replace(" ']", "").split(" ', ' "))
    
# формат Series для оценки
tags = pd.Series(tag_list)

# преобразование признака с тэгами в удобный формат
hotels['tags'] = hotels['tags'].apply(
    lambda x: x.replace("[' ", "").replace(" ']", "").split(" ', ' ")
)

* Создание бинарных признаков на основе тэгов

In [8]:
# поездка с целью отдыха
hotels['leisure'] = hotels['tags'].apply(
    lambda x: 1 if 'Leisure trip' in x else 0
)

# деловая поездка
hotels['business'] = hotels['tags'].apply(
    lambda x: 1 if 'Business trip' in x else 0
)

# путешевствует один
hotels['lone'] = hotels['tags'].apply(
    lambda x: 1 if 'Solo traveler' in x else 0
)

# путешевствует пара
hotels['couple'] = hotels['tags'].apply(
    lambda x: 1 if 'Couple' in x else 0
)

# путешевствует семья
hotels['family'] = hotels['tags'].apply(
    lambda x: 1 if ('Family with young children' in x) or (
        'Family with older children' in x) else 0
)

# путешевствует группа
hotels['group'] = hotels['tags'].apply(
    lambda x: 1 if 'Group' in x else 0
)

# со смартфона
hotels['smartphone'] = hotels['tags'].apply(
    lambda x: 1 if 'Submitted from a mobile device' in x else 0
)

# функция для поиска по тэгам
def lux(x):
    expression = re.compile(
    "(?i)(\W|^)(Grand|Premier|View|Superior|Marvellous|Spa|Pool|Luxe|Deluxe|King|Queen|Lounge|Balcony)(\W|$)"
    )    
    if list(filter(expression.search, x)): return 1
    else: return 0
    
    
# класс номера
hotels['lux'] = hotels['tags'].apply(lux)

# функция для определения времени пребывания
def stay(x):
    z = [y for y in x if y.startswith('Stayed')]
    if z: return int(z[0].split()[1])
    else: return 0

# время пребывания в отле (ночей)
hotels['stayed'] = hotels['tags'].apply(stay)

* Получение недостающих координат

In [9]:
# адреса отелей без привязки к координатам
addr = hotels[
    pd.isna(hotels.lat)
    ]['hotel_address'].value_counts().index

geolocator_arcgis = ArcGIS()

# цикл присваивает координаты по адресам
for i in addr:
    lat = round(geolocator_arcgis.geocode(i).latitude, 6)
    long = round(geolocator_arcgis.geocode(i).longitude, 6)
    hotels.loc[hotels['hotel_address'] == i, 'lat'] = lat
    hotels.loc[hotels['hotel_address'] == i, 'lng'] = long

* Признак дистанция от отеля до центра города

In [10]:
# функция для получения дистанции от отеля до центра города
def distance(row):
    if row['hotel_country'] == 'United Kingdom':
        return geodesic((51.500322, -0.140519), (row.lat, row.lng))
    if row['hotel_country'] == 'France':
        return geodesic((48.858174, 2.294383), (row.lat, row.lng))
    if row['hotel_country'] == 'Netherlands':
        return geodesic((52.373277, 4.892011), (row.lat, row.lng))
    if row['hotel_country'] == 'Italy':
        return geodesic((45.464528, 9.188393), (row.lat, row.lng))
    if row['hotel_country'] == 'Austria':
        return geodesic((48.209213, 16.370674), (row.lat, row.lng))
    else:
        return geodesic((41.386620, 2.170433), (row.lat, row.lng))
    

# применение функции и получение нового признака
hotels['distance'] = hotels.apply(lambda row: distance(row), axis=1)

# преобразование geodesic типа данных во float
hotels['distance'] = hotels['distance'].apply(
    lambda x: round(float(str(x).split()[0]), 4)
    )

* пересчёт слов в отзыве

In [11]:
hotels['review_total_negative_word_counts'] = hotels[
    'negative_review'
    ].apply(lambda x: len(x.split()))

hotels['review_total_positive_word_counts'] = hotels[
    'positive_review'
    ].apply(lambda x: len(x.split()))

* Работа с текстовыми признаками отзывы

In [12]:
# удаление лишних пробелов и приведение строки признака к нижнему регистру
hotels['negative_review'] = hotels['negative_review'].apply(
    lambda x: x.strip().lower()
)

hotels['positive_review'] = hotels['positive_review'].apply(
    lambda x: x.strip().lower()
)


# функция определения компаунда для негативных отзывов
def negative_compound(row):
    if row.review_total_negative_word_counts == 0: return 0
    elif row.review_total_negative_word_counts == 1:
        reg_1 = re.compile(("(?i)(\W|^)(nothing|none|nil|na|no|non)(\W|$)"))
        if reg_1.search(row.negative_review): return 0
        else: return row.negative_scores['compound']
    elif row.review_total_negative_word_counts == 2:
        reg_2 = re.compile(("(?i)(\W|^)(no negative|n a|nothing really|no complaints|absolutely nothing|not applicable|no negatives|no problems)(\W|$)"))
        if reg_2.search(row.negative_review): return 0
        else: return row.negative_scores['compound']
    elif row.review_total_negative_word_counts == 3:
        reg_3 = re.compile(("(?i)(\W|^)(nothing at all|nothing to dislike|nothing in particular|nothing to report|nothing to mention|nothing to say|nothing to complain)(\W|$)"))
        if reg_3.search(row.negative_review): return 0
        else: return row.negative_scores['compound']
    else: return row.negative_scores['compound']


# функция определения компаунда для позитивных отзывов
def positive_compound(row):
    if row.review_total_positive_word_counts == 0: return 0
    elif row.review_total_positive_word_counts == 1:
        reg_1 = re.compile(("(?i)(\W|^)(nothing|none|nil|na|no|non)(\W|$)"))
        if reg_1.search(row.positive_review): return 0
        else: return row.positive_scores['compound']
    elif row.review_total_positive_word_counts == 2:
        reg_2 = re.compile(("(?i)(\W|^)(no positive)(\W|$)"))
        if reg_2.search(row.positive_review): return 0
        else: return row.positive_scores['compound']
    else: return row.positive_scores['compound']
  

# класс анализатора эмоциональной окраски текста
sia = SentimentIntensityAnalyzer()


# создание признаков эмоциональной окраски текста для
# негативных и позитивных отзывов
hotels['negative_scores'] = hotels['negative_review'].apply(lambda x: sia.polarity_scores(x))
hotels['positive_scores'] = hotels['positive_review'].apply(lambda x: sia.polarity_scores(x))


# создание признака с баллами компаунд для для негативных и позитивных отзывов
hotels['neg_compound'] = hotels.apply(negative_compound, axis=1)
hotels['pos_compound'] = hotels.apply(positive_compound, axis=1)

* выделение порядкового номера дня в году в отдельный бинарный признак

In [13]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'])

hotels['day'] = hotels['review_date'].dt.day_of_year

binary_enc = ce.BinaryEncoder(cols=['day'])
df = binary_enc.fit_transform(hotels['day'])
hotels = pd.concat([hotels, df], axis=1)

* удаление текстовых признаков

In [14]:
cols_to_drop = ['lat', 'lng', 'review_date']

for i in hotels.columns:
    if hotels[i].dtype == 'object':
        cols_to_drop.append(i)
        

hotels.drop(cols_to_drop, axis=1, inplace=True)

* проверка данных на нормальность

In [15]:
# уровень значимости
alpha = 0.05

In [16]:
# цикл распределит столбцы на нормальные и не нормальные
# в зависимости от полученного p-value
norma = []
not_norma = []

for i in hotels.columns:
    _, p = stats.shapiro(hotels[i])
    if p <= alpha: not_norma.append(i)
    else: norma.append(i)

# model

In [17]:
#X = hotels.drop(['reviewer_score'], axis = 1)  
#y = hotels['reviewer_score']

In [18]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [19]:
# Создаём модель  
#regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
#regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
#y_pred = regr.predict(X_test)

In [20]:
#print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [55]:
df = pd.da